 # MTH8408 : Méthodes d'optimisation et contrôle optimal
 ## Laboratoire 6: Optimisation avec contraintes et contrôle optimal
Tangi Migot, Dominique Orban et Paul Raynaud

Dans ce lab, on va utiliser le module [OptimalControl.jl](https://github.com/control-toolbox/OptimalControl.jl) afin de modéliser les problèmes de contrôle optimal.
L'installation de ce module nécessite la commande suivante:

In [ ]:
using Pkg
Pkg.Registry.add(RegistrySpec(url = "https://github.com/control-toolbox/ct-registry.git"))

L'installation des modules peut-être assez longue, profitez-en pour lire la [documentation de OptimalControl.jl](https://control-toolbox.org/OptimalControl.jl/stable/index.html).

De plus, nous allons utiliser une version en cours de développement de CTDirect, qui est une dépendance de OptimalControl.jl.
Cette version contient une fonctionnalité permettant la création d'un ADNLPModel à partir du problème de contrôle discretisé.
Pour installer la version développée dans la branche `export` du module CTDirect.jl, il faut utiliser l'instruction suivante :

In [ ]:
Pkg.add(url="https://github.com/control-toolbox/CTDirect.jl.git#export")

In [ ]:
Pkg.add(["OptimalControl", "ADNLPModels", "NLPModels", "Percival", "Plots"]) # les autres modules nécessaires au laboratoire

Si jamais votre installation de `OptimalControl` échoue, mettez à jour julia:
```julia
using UpdateJulia # à ajouter au préalable
update_julia() # puis on redémarre VS Code
```

## Exercice 1: Commande optimale

Dans cet exercice, on considère le problème de gestion de portefeuille vu en cours:
$$
\max_{x,u} \int_0^T (1-u(t))x(t) dt, \quad x(0)=x_0, \dot{x}(t) = \gamma u(t) x(t), \quad 0 \leq u(t) \leq 1
$$
modélisé à l'aide de `OptimalControl.jl`.
Compléter les `...` en vous inspirant du [tutoriel de OptimalControl.jl](https://control-toolbox.org/OptimalControl.jl/stable/tutorial-goddard.html).
On suppose `n = 100`, `T = 1`, `γ = 3` et `x0 = 0.1`.

In [ ]:
T = ...
γ = ...
x0 = ...

@def ocp begin
  tf ∈ R, variable
  t ∈ [ 0, ... ], time
  x ...
  u ...
  ... ≤ u(t) ≤ ...
  
  x(0) == ...
  ẋ(t) == ... # 

  ∫(...) → ...
end

n = ...
docp = DirectTranscription(ocp, grid_size=n) 

### Question 1: Résoudre et affichage
Lire la documentation de OptimalControl.jl et résoudre le problème discretisé `docp`.

In [ ]:
sol = ...(docp) # résout le problème via Ipopt

À partir de la documentation, affichez `u` et `x` (en autres).

In [ ]:
fig = ...(sol)

La figure peut-être enregistrée avec la fonction:

In [ ]:
using Plots
savefig(fig, "Mon-nom-de-fichier.pdf")

Donner une interprétation des résultats obtenus.

### Conversion en un ADNLPModel

On peut obtenir un ADNLPModel à partir de `docp`.

In [ ]:
nlp = getNLP(docp) # récupère le problème discretisé sous forme d'un ADNLPModel

Pour minimiser le `nlp` modèle (contraint) extrait, on peut utiliser le solver de [Percival.jl](https://github.com/JuliaSmoothOptimizers/Percival.jl). 

In [ ]:
ges = percival(nlp, verbose = 0) # modifier verbose=1 pour obtenir un affichage des itérations
@test :first_order == stats.status

Pour afficher `x` et `u`, il est cependant plus simple de réutiliser les fonctionnalités de CTDirect.jl:

In [ ]:
sol = CTDirect._OptimalControlSolution(ges, docp)
fig = plot(sol)

Ce n'est pas une erreur de votre implémentation si les états adjoints `p` sont inversés.

### Question 2: Bang-bang
A quelle instant `t` se situe le changement de trajectoire?

Modifier `γ` afin d'obtenir un contrôle non bang-bang, i.e. `ẋ` n'est pas linéaire en `u`.

In [ ]:
...

### Question 3: Un contrôle non bang-bang
Proposer une modification continue de ẋ de manière à déterminer un contrôle qui n'est plus bang-bang.
Afficher les graphiques obtenus.
`T, x0` et `n` sont identiques à précédemment.

In [ ]:
T = ...
# γ = ...
x0 = ...

@def ocp begin
  tf ∈ R, variable
  t ∈ [ 0, ... ], time
  x ...
  u ...
  ... ≤ u(t) ≤ ...
  
  x(0) == ...
  ẋ(t) == ... # le nouveau γ peut être défini ici directement, et ainsi varier en fonction de t, i.e. γ(t)

  ∫(...) → max
end

n = ...
docp = DirectTranscription(ocp, grid_size=n) 

In [ ]:
sol = ...(docp) # résout le problème via Ipopt
...(sol)

## Exercice 2: Le réservoir

Dans ce dernier exercice, on considère le problème de réservoir (exercice 5 control.pdf):.

$$
\max_{x_1,x_2,u} x_2(T), \quad \dot{x_1} = -x_1 + u, \dot{x_2} = x_1, x_1(0) = x_2(0) = 0, x_1(1)=γ, x_1(t)\leq γ
$$
Compléter le code suivant en considérant $n=10$ et $γ = 0.5$.

In [ ]:
T = ...
γ = ...

@def ocp begin
  tf ∈ R, variable
  t ∈ [0, ...], time
  x ...
  u ...
  ... ≤ u(t) ≤ ...
  
  x(0) == [...]
  ẋ(t) == [...]
  x₁(T) == ...

  ... → ...
end

n = ...
docp = DirectTranscription(ocp, grid_size=n)
nlp = getNLP(docp)

Résoudre et afficher le problème

In [ ]:
sol = ...(docp) # résout le problème via Ipopt
...(sol)

In [ ]:
nlp = getNLP(docp)
sol = CTDirect._OptimalControlSolution(ges, docp)
fig = plot(sol)

### Question: Convergence en n

Recommencer le processus pour observer la convergence du contrôle pour plusieurs valeurs de n (100, 500, 1000).

## Exercice 3: Git/Github, Pull request, OptimizationProblems

L'objectif de cet exercice est d'intégrer le problème que vous avez développé dans la question 3 (Un contrôle non bang-bang) au dépôt https://github.com/paraynaud/OptimizationProblems.jl en ouvrant une "Pull Request".

A tout moment, n'hésitez pas à demander de l'aide sur Zulip sur cette partie.

### Etape 1: Clone & fork (similaire aux laboratoires précédents)

Afin de faire une proposition de modification au package `paraynaud/OptimizationProblems.jl` vous allez devoir "cloner" ce package sur votre compte github et ouvrir une nouvelle branche où ajouter la modification.

- [ ] Aller sur https://github.com/paraynaud/OptimizationProblems.jl et cliquer sur "Fork" en haut à droite de l'écran.
- [ ] En suivant les indications du README du lab1 et clonez la version sur votre compte github.
- [ ] En suivant les inditions du README du lab2 et ouvrez une nouvelle branche de travail. (en général on essaye d'éviter de travailler directement sur la branche `main`).

### Etape 2: Compléter l'implémentation des problèmes

Lisez la documentation du package OptimizationProblems.jl, https://juliasmoothoptimizers.github.io/OptimizationProblems.jl/dev/contributing/, particulièrement la section "Templates for the new functions".

Pour créer votre problème, ajouter un fichier `<votre_nom>.jl` dans le sous-répertoire `src/ADNLPProblems/`.
Dans ce fichier, vous devez ajouter la formulation de votre problème de la question 3, en conservant `n` comme un argument optionnel, mis à `10` par défaut.

Il est fort probable qu'en plus de votre fichier `src/ADNLPProblems/<votre_nom>.jl`, vous modifiez le fichier `Project.toml` lors de l'ajout des modules OptimalControl.jl et CTDirect.jl, ce n'est pas un problème.

### Etape 3: Modifier le package et mise en ligne
- [ ] Une fois les modifications satisfaisantes faire un `git push origin nom_de_votre_branch` pour mettre en ligne vos modifications.
- [ ] Ouvrir la Pull Request vers paraynaud/OptimizationProblems.jl **et non JuliaSmoothOptimizers/OptimizationProblems.jl** !!!